# Exlplaination about this notebook

In this notebook I gathered the Date, Headline and the news text from the links. 
The links where gathered in notebook `get_links.ipynb`!

**Caution!** This code will take about 24:30h. 

After gathering all the text from the news website I translated the text to english, beacause the models that I used did not provided german sentimant analysis.

In [1]:
import requests
import pandas as pd
import re
from bs4 import BeautifulSoup
import selenium as se
from time import sleep
from tqdm import tqdm as tq

# Driver for Firefox, Chrome, Edge, etc.
from selenium import webdriver

# Mode of locating html elements: ID, CSS_SELECTOR, XPATH, ...
from selenium.webdriver.common.by import By               

# Using specific keyboard keys like ENTER, ESCAPE, ...
from selenium.webdriver.common.keys import Keys

# Methods for dropdown
from selenium.webdriver.support.select import Select

import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException


In [2]:
article_df = pd.read_csv('..\\data\\links\\article_list.tsv', sep='\t', header=0)
article_df

,links
0,/nachricht/aktien/aktien-frankfurt-eroeffnung-...
1,/nachricht/aktien/aktien-frankfurt-ausblick-da...
2,/nachricht/aktien/10-vor-9-boersenhandel-am-do...
3,/nachricht/aktien/xetra-handel-dax-praesentier...
4,/nachricht/aktien/aktien-frankfurt-schluss-dax...
...,...
16998,/nachricht/aktien/aktien-frankfurt-schluss-wal...
16999,/nachricht/aktien/boerse-frankfurt-news-zertif...
17000,/nachricht/aktien/boerse-frankfurt-news-markts...
17001,/nachricht/aktien/aktien-frankfurt-dax-und-mda...


In [3]:
# Use the Firefox browser to open the URL
driver = webdriver.Firefox()

In [4]:
def get_news_from_article(driver, article_list):
    i = 0
    df = pd.DataFrame(columns=['date', 'headline', 'news_text', 'article_url'])
    base_url = r'https://www.finanzen.net'

    for article in tq(article_list.loc[:, 'links'], desc='Articles'):
        article_url = base_url + article
        driver.get(article_url)
        source = driver.page_source
        soup = BeautifulSoup(source, 'html.parser')

        # Find date
        date = 'None'
        dates = soup.find_all('div', class_='grid__item-12')
        for date_tag in dates:
            if 'Uhr' in date_tag.text:
                date = date_tag.text
                break

        # Find news text
        news_text = 'None'
        news_container = soup.find_all('div', class_='news-container__text')
        if news_container:
            news_list = [news.get_text(strip=True) for news in news_container]
            news_text = ' '.join(news_list)  # Combine all paragraphs into one string
        
        # Find headline
        headline = 'None'
        headlines = soup.find_all('h1')
        if headlines:
            headline = headlines[0].get_text(strip=True)
        
        try:
            # Add to DataFrame (wrap scalar values in lists)
            df = pd.concat([df, pd.DataFrame({'date': [date], 
                                              'headline': [headline], 
                                              'news_text': [news_text],
                                              'article_url': [article_url]})], ignore_index=True)
        except ValueError as e:
            print(f'Error: {str(e)}')
            print(f'news_text: {news_text}')
            print(f'Script stops at article {article}')
            print(f'URL: {article_url}')
            print(f'headline: {headline}')
            print(f'date: {date}')
            break
        
        # Drop duplicates
        df.drop_duplicates(subset='news_text', inplace=True)
        sleep(1)  # Throttle scraping to avoid bans

        i += 1
        if i == 500:
            break

    return df



In [5]:
df = get_news_from_article(driver=driver, article_list=article_df)

Articles:   3%|▎         | 499/17003 [24:37<13:34:29,  2.96s/it]


In [6]:

n = len(df)

# save the data
df.to_csv(f'..\\data\\DAX-News_n={n}.csv', index=True)


To continue reading please look at the notebook: `translate_news.ipynb`